In [1]:
# %load honest_net.py
import numpy as np
import pandas as pd

df = pd.read_csv("../../data/honest_net.tsv", sep="\t")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]


# C&P from https://stackoverflow.com/a/39513799
def gini(x):
    # (Warning: This is a concise implementation, but it is O(n**2)
    # in time and memory, where n = len(x).  *Don't* pass in huge
    # samples!)

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad / np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g


def parse_array(s):
    try:
        return np.fromstring(s, dtype=float, sep="|")
    except TypeError:
        return np.array([float("nan")])


def expand(row):
    compute = parse_array(row.compute)
    weakest = np.argmin(compute)
    strongest = np.argmax(compute)
    d = {}

    def wsg(k, v):
        d[k + "_weakest"] = v[weakest]
        d[k + "_strongest"] = v[strongest]
        d[k + "_gini"] = gini(v)

    rcompute = compute / np.sum(compute)
    wsg("compute", rcompute)
    activations = parse_array(row.activations)
    assert np.sum(activations) == row.number_activations or row.error
    ractivations = activations / np.sum(activations)
    wsg("activations", ractivations)
    reward = parse_array(row.reward)
    rreward = reward / np.sum(reward)
    wsg("reward", rreward)
    efficiency = rreward / ractivations
    wsg("efficiency", efficiency)
    d["activations_compute_gini_delta"] = d["activations_gini"] - d["compute_gini"]
    d["reward_activations_gini_delta"] = d["reward_gini"] - d["activations_gini"]
    return d


df = df.join(df.apply(expand, axis=1, result_type="expand"))

## Incentive Schemes for Georges  Protocol

In [2]:
d = df[df.protocol == "george"]

### Efficiency of the weakest miner

We pick the weakest miner by compute power and calculate its

efficiency = relative reward / relative activations

In [3]:
eff = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="efficiency_weakest",
)
eff.to_latex(buf="../../tex/george-reward-inequality/tab/efficiency-weakest.tex")
eff

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.992089  0.992089  0.992089  0.992089  0.992089
               2    0.979370  0.984524  0.984524  0.984524  0.984524
               4    0.969144  0.982597  0.979192  0.968015  0.968317
               8    0.950177  0.985601  0.979427  0.943237  0.945423
               16   0.874490  0.982092  0.975922  0.878651  0.880480
               32   0.786187  0.979611  0.975540  0.764662  0.765299
               64   0.651540  0.977193  0.971718  0.550990  0.554872
               128  0.393089  0.973449  0.970478  0.295943  0.296561
60.0           1    0.997408  0.997408  0.997408  0.997408  0.997408
               2    0.984510  0.992090  0.992090  0.992090  0.992090
               4    0.982581  0.991625  0.989028  0.984381  0.985651
               8    0.966535  0.989153  0.986620  0.968289  0.968482
               16   0.928676  0.990979  0.987295  0.942620  0.944108
               32   0.925923  0.991585  0.989370  0.876803  0.876778
               64   0.768543  0.989375  0.987041  0.749814  0.751270
               128  0.582835  0.989702  0.987435  0.555415  0.556818
120.0          1    0.998914  0.998914  0.998914  0.998914  0.998914
               2    1.014695  0.995182  0.995182  0.995182  0.995182
               4    1.004402  0.996496  0.994990  0.991689  0.992175
               8    1.013124  0.996954  0.996247  0.988202  0.987836
               16   0.954099  0.995964  0.993830  0.966080  0.966445
               32   0.910821  0.996097  0.994343  0.937632  0.937301
               64   0.845134  0.995496  0.994583  0.878910  0.878869
               128  0.777721  0.994727  0.993366  0.755060  0.754941
300.0          1    0.999150  0.999150  0.999150  0.999150  0.999150
               2    1.001489  0.998209  0.998209  0.998209  0.998209
               4    0.988634  0.997473  0.997453  0.996140  0.995720
               8    0.999812  0.998644  0.998050  0.994555  0.994648
               16   1.004682  0.996407  0.995822  0.983355  0.983204
               32   1.033485  0.997694  0.997061  0.973672  0.973663
               64   0.971035  0.997727  0.996949  0.950028  0.950367
               128  0.895926  0.998492  0.998033  0.907276  0.907244
600.0          1    0.999739  0.999739  0.999739  0.999739  0.999739
               2    1.009494  0.999407  0.999407  0.999407  0.999407
               4    1.004113  0.999346  0.999459  0.998562  0.998153
               8    1.011927  0.999362  0.998755  0.996917  0.997235
               16   0.989626  0.999823  0.999448  0.996420  0.996605
               32   1.008185  0.999390  0.999114  0.989924  0.989915
               64   1.012456  0.998201  0.997779  0.971146  0.971101
               128  1.021848  0.999294  0.999191  0.945918  0.945555

### Reward Gini Fairness

We compare the rewards of all miners using the Gini coefficient. A lower coefficient implies more fairness. All numbers should be compared to the Gini coefficient of the compute powers.

In [4]:
d.compute_gini.unique()

array([0.3])

In [5]:
gini = d.rename(
    columns={'incentive_scheme': 'Incentive Scheme',
            'block_interval': 'Block Interval'}
).pivot(
    index=["Block Interval", "k"],
    columns=["Incentive Scheme"],
    values="reward_gini",
)
gini.to_latex(buf="../../tex/george-reward-inequality/tab/reward-gini.tex")
gini

Incentive Scheme       block  constant  discount    hybrid    punish
Block Interval k                                                    
30.0           1    0.301341  0.301341  0.301341  0.301341  0.301341
               2    0.303348  0.302833  0.302833  0.302833  0.302833
               4    0.305474  0.303605  0.304212  0.305758  0.305595
               8    0.311103  0.303932  0.304862  0.311636  0.311487
               16   0.322439  0.303836  0.304951  0.323946  0.323647
               32   0.342107  0.303925  0.304895  0.348516  0.348205
               64   0.373283  0.304753  0.305837  0.396385  0.395536
               128  0.439060  0.307100  0.308067  0.477832  0.476695
60.0           1    0.300484  0.300484  0.300484  0.300484  0.300484
               2    0.301437  0.301341  0.301341  0.301341  0.301341
               4    0.302621  0.302038  0.302462  0.303290  0.303116
               8    0.303222  0.302084  0.302517  0.305669  0.305641
               16   0.312242  0.302239  0.302846  0.312219  0.312138
               32   0.323930  0.301933  0.302513  0.324570  0.324422
               64   0.345049  0.301506  0.302067  0.349006  0.348772
               128  0.378968  0.302765  0.303291  0.398581  0.398123
120.0          1    0.300198  0.300198  0.300198  0.300198  0.300198
               2    0.301106  0.301277  0.301277  0.301277  0.301277
               4    0.301921  0.301307  0.301461  0.301842  0.301811
               8    0.303194  0.301422  0.301693  0.303501  0.303474
               16   0.306629  0.301202  0.301490  0.305819  0.305788
               32   0.314912  0.301337  0.301673  0.312327  0.312289
               64   0.320127  0.300274  0.300563  0.323564  0.323536
               128  0.338929  0.300543  0.300864  0.349365  0.349268
300.0          1    0.299957  0.299957  0.299957  0.299957  0.299957
               2    0.300812  0.300929  0.300929  0.300929  0.300929
               4    0.301003  0.300958  0.301013  0.301146  0.301136
               8    0.301849  0.300814  0.300894  0.301467  0.301466
               16   0.300827  0.300614  0.300761  0.302722  0.302694
               32   0.302891  0.301317  0.301466  0.305580  0.305564
               64   0.305872  0.300660  0.300793  0.310003  0.309993
               128  0.312490  0.300015  0.300183  0.318649  0.318575
600.0          1    0.299879  0.299879  0.299879  0.299879  0.299879
               2    0.300703  0.300752  0.300752  0.300752  0.300752
               4    0.300425  0.300764  0.300815  0.300915  0.300897
               8    0.299081  0.300844  0.300920  0.301272  0.301246
               16   0.302709  0.300886  0.300978  0.301950  0.301920
               32   0.306438  0.300681  0.300749  0.302828  0.302824
               64   0.304365  0.300474  0.300554  0.305035  0.305023
               128  0.305547  0.300142  0.300204  0.309960  0.309965